## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [4]:
!brew install postgresql


==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 5 taps (heroku/brew, shivammathur/php, coursier/formulas, homebrew/core and homebrew/cask).
==> New Formulae
action-docs                langgraph-cli              sf
ansible-builder            lbfgspp                    sleek
boring                     libmsquic                  surfer
carapace                   libspelling@0.2            termscp
clickhouse-sql-parser      localai                    tex-fmt
decasify                   markdown-oxide             toml2json
dipc                       minijinja-cli              vipsdisp
distill-cli                node@22                    wasi-libc
fcft                       pytest                     wasi-runtimes
foot                       python-freethreading       wasm-component-ld
hawkeye                    rec

In [5]:
!pip install wrds

  Using cached wrds-3.2.0-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of wrds to determine which version is compatible with other requirements. This could take a while.
  Using cached wrds-3.1.6-py3-none-any.whl.metadata (4.5 kB)
  Using cached psycopg2-binary-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2-binary: filename=psycopg2_binary-2.9.10-cp38-cp38-macosx_10_9_x86_64.whl size=143865 sha256=9efef57799e02bd074f983f118125a3efc648205c7c64f6b13076794bd9b0310
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/df/fb/7d/c43b1faf8e1ae26e14d1e183eca11ef5560125067a34948080
Successfully built psycopg2-binary

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [7]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [8]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [9]:
sp500.head()

,permno,start,ending,date,ret
0,77178,1999-07-22,2023-12-29,2000-01-31,-0.278921
1,48485,1976-07-01,2006-03-31,2000-01-31,-0.139194
2,19502,1979-10-11,2023-12-29,2000-01-31,-0.044872
3,28847,1986-12-18,2002-05-14,2000-01-31,-0.311891
4,56573,1986-02-13,2023-12-29,2000-01-31,-0.134135


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [10]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [11]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
39745,PRESTON MINES LTD,None,1962-07-02,1968-01-01,35588,10,2,1081,PRS
78642,LAFAYETTE INDUSTRIES INC,50637710,1995-07-13,1996-05-08,80938,11,3,2542,LAFI
72228,SAPIENS INTERNATIONAL CORP N V,N7716A10,1992-05-29,1995-07-19,78759,12,3,7372,SPNSF
93645,INTERSTATE HOTEL CORP,46088R10,1999-06-18,2002-07-31,86973,11,3,7010,IHCO
64298,FUTURE GERMANY FUND INC,36090510,1990-02-28,1995-06-29,76083,14,1,6722,FGF


In [12]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
108839,41080,1965-01-07,2023-12-29,2002-05-31,-0.127850,CUMMINS INC,23102110,2002-01-02,2003-12-09,11,1,3519,CUM
922870,60599,1999-03-25,2023-03-17,2020-04-30,0.122622,CENTURYLINK INC,15670010,2014-02-13,2020-09-17,11,1,4812,CTL
1028182,89269,2016-03-30,2023-12-29,2022-10-31,0.094075,CENTENE CORP DEL,15135B10,2014-01-29,2023-12-29,11,1,6324,CNC
6894,64995,1994-03-02,2023-12-29,2000-02-29,-0.180119,KEYCORP NEW,49326710,1994-03-02,2002-01-01,11,1,6021,KEY
1009078,76095,2016-03-30,2023-12-29,2022-04-29,-0.062874,HOLOGIC INC,43644010,2018-01-29,2023-12-29,11,3,3844,HOLX


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [8]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [15]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
42113,48274.0,1989-08-03,2011-12-12,2005-05-31,0.068434,NICOR INC,65408610,2004-06-10,2011-12-09,11.0,1.0,4924.0,GAS,007904,01,LC,P,1969-05-02,2011-12-30 00:00:00.000000
21022,27887.0,1972-09-14,2019-12-31,2002-08-30,-0.090704,BAXTER INTERNATIONAL INC,07181310,2002-02-01,2004-02-16,11.0,1.0,3841.0,BAX,002086,01,LC,P,1962-01-31,2020-10-21 14:26:10.563769
81974,90829.0,2008-08-27,2019-12-31,2010-08-31,0.140535,C F INDUSTRIES HOLDINGS INC,12526910,2005-08-11,2019-12-31,11.0,1.0,2873.0,CF,163946,01,LC,P,2005-08-11,2020-10-21 14:26:10.563769
39158,13856.0,1957-03-01,2019-12-31,2004-12-31,0.050491,PEPSICO INC,71344810,2004-06-10,2017-12-19,11.0,1.0,2086.0,PEP,008479,01,LC,P,1962-01-31,2020-10-21 14:26:10.563769
137326,71175.0,1994-03-11,2019-12-31,2018-02-28,-0.041925,UNUM GROUP,91529Y10,2007-03-02,2019-09-11,11.0,1.0,6321.0,UNM,012726,01,LU,P,1986-12-31,2020-10-21 14:26:10.563769


In [16]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
49267,2006-04-28,86102.0,FEDERATED INVESTORS INC PA,31421110,11.0,1.0,6282.0,FII,066731,01,2003-04-16,2012-12-31,-0.101152
120416,2015-11-30,81910.0,INVESCO LTD,G491BT10,12.0,1.0,6282.0,IVZ,029804,02,2008-08-21,2019-12-31,0.023817
112968,2014-11-28,78975.0,INTUIT INC,46120210,11.0,3.0,7370.0,INTU,027928,01,2000-12-11,2019-12-31,0.066583
19214,2002-05-31,64186.0,C I G N A CORP,12550910,11.0,1.0,6324.0,CI,002547,01,1982-04-08,2019-12-31,-0.027064
134250,2017-09-29,15707.0,HEWLETT PACKARD ENTERPRISE CO,42824C10,11.0,1.0,7374.0,HPE,026156,01,2015-11-02,2019-12-31,0.046962


In [21]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
44,2003-09-30,500
212,2017-09-29,501
84,2007-01-31,500
39,2003-04-30,500


In [22]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
113348,2014-11-28,26825.0,KELLOGG CO,48783610,11.0,1.0,2043.0,K,006375,01,1961-04-26,2019-12-31,0.043465
507,2000-01-31,80599.0,LEHMAN BROTHERS HOLDINGS INC,52490810,11.0,1.0,6282.0,LEH,030128,01,1998-01-12,2008-09-16,-0.157934
60387,2007-10-31,40272.0,INTERNATIONAL FLAVORS & FRAG INC,45950610,11.0,1.0,2869.0,IFF,006078,01,1976-03-04,2019-12-31,-0.012297
50310,2006-06-30,78034.0,PATTERSON COMPANIES INC,70339510,11.0,3.0,5990.0,PDCO,025880,01,2005-10-11,2018-03-16,0.019854
97932,2012-10-31,16600.0,HERSHEY CO,42786610,11.0,1.0,2066.0,HSY,005597,01,1957-03-01,2019-12-31,-0.028777
70840,2009-02-27,24766.0,NORTHROP GRUMMAN CORP,66680710,11.0,1.0,3812.0,NOC,007985,01,1985-06-13,2019-12-31,-0.215295
23639,2002-12-31,13119.0,MAYTAG CORP,57859210,11.0,1.0,3633.0,MYG,007139,01,1960-12-14,2006-03-31,-0.079457
112012,2014-09-30,28345.0,MURPHY OIL CORP,62671710,11.0,1.0,1382.0,MUR,007620,01,2005-08-15,2017-07-25,-0.089003
93624,2012-03-30,52329.0,JACOBS ENGINEERING GROUP INC,46981410,11.0,1.0,8711.0,JEC,006216,01,2007-10-26,2019-12-31,-0.040026
114295,2015-01-30,91461.0,WESTERN UNION CO,95980210,11.0,1.0,6099.0,WU,175263,01,2006-10-02,2019-12-31,-0.050810


Save to CSV

In [13]:
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
1048620,16678,1957-03-01,2023-12-29,2023-04-28,-0.014989,KROGER CO,50104410,2009-12-08,2023-12-29,11,1,5411,KR
1077758,24328,2022-10-03,2023-12-29,2023-12-29,-0.032533,E Q T CORP,26884L10,2017-12-12,2023-12-29,11,1,1311,EQT
752177,60943,1999-10-18,2021-12-17,2016-06-30,0.023677,LEGGETT & PLATT INC,52466010,2004-06-10,2017-01-10,11,1,2515,LEG
311197,13856,1957-03-01,2023-12-29,2006-10-31,-0.027889,PEPSICO INC,71344810,2004-06-10,2017-12-19,11,1,2086,PEP
976286,77605,1995-02-24,2023-12-29,2021-07-30,0.066417,BOSTON SCIENTIFIC CORP,10113710,2004-06-10,2023-12-29,11,1,3841,BSX


In [14]:
sp500_full.to_csv('sp500_constituents.csv')

In [16]:
import yfinance as yf


# Function to get CIK by ticker
def get_cik_from_yfinance(tickers):
    cik_dict = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            cik = stock.info.get('CIK')
            cik_dict[ticker] = cik
            print(f"Ticker: {ticker}, CIK: {cik}")
        except Exception as e:
            print(f"Could not retrieve CIK for ticker {ticker}: {e}")
    return cik_dict

# Example usage with a list of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL']
cik_dict = get_cik_from_yfinance(tickers)


Ticker: AAPL, CIK: None
Ticker: MSFT, CIK: None
Ticker: GOOGL, CIK: None
